In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-06-13 06:46:53--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.07s   

2025-06-13 06:46:53 (375 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [6]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})


         isbn                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  
0    Mark P. O. Morford  
1  Richard Bruce Wright  
2          Carlo D'Este  
3      Gina Bari Kolata  
4       E. J. W. Barber  
     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
4  276729  0521795028     6.0


In [8]:
# add your code here - consider creating a new cell for each section of code
# For the model to work effectively, we need to filter out noise.
# We will remove users who have rated fewer than 200 books and
# books that have been rated by fewer than 100 users.

# Get user and book rating counts
user_rating_counts = df_ratings['user'].value_counts()
book_rating_counts = df_ratings['isbn'].value_counts()

# Filter users with at least 200 ratings
active_users = user_rating_counts[user_rating_counts >= 200].index
df_ratings_filtered = df_ratings[df_ratings['user'].isin(active_users)]

# Filter books with at least 100 ratings
popular_books = book_rating_counts[book_rating_counts >= 100].index
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(popular_books)]

# Merge the filtered ratings with the books dataframe to get book titles
df_merged = pd.merge(df_ratings_filtered, df_books, on='isbn')

# Remove any duplicates where a user might have rated the same book title more than once
df_merged = df_merged.drop_duplicates(['user', 'title'])

# --- Create User-Item Matrix ---

# We will create a pivot table where rows are book titles, columns are users,
# and values are the ratings. This matrix will be used to calculate similarity.
# We fill missing values (NaN) with 0, indicating a user has not rated a book.
book_pivot = df_merged.pivot(index='title', columns='user', values='rating').fillna(0)

# Convert the pivot table into a sparse matrix for memory efficiency.
book_pivot_sparse = csr_matrix(book_pivot.values)

# --- Build the K-Nearest Neighbors Model ---

# We use NearestNeighbors with the 'cosine' metric, which is effective for
# this kind of data. 'brute' algorithm will compute distances for all pairs.
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_pivot_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [11]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    # Get the data for the specified book from the sparse matrix
    # The model returns the 6 nearest neighbors (including the book itself)
    distances, indices = model_knn.kneighbors(book_pivot.loc[[book]].values.reshape(1, -1), n_neighbors=6)

    # Prepare the list for recommendations
    recommended_books = []

    # The result is a 2D array, so we iterate through the first row
    # We start from index 1 to skip the input book itself.
    for i in range(1, len(distances.flatten())):
        # Get the title of the recommended book using its index
        title = book_pivot.index[indices.flatten()[i]]
        # Get the distance (lower is more similar)
        distance = distances.flatten()[i]
        recommended_books.append([title, distance])

    # The test case expects the recommendations to be in a specific order,
    # from largest distance to smallest. We reverse the list to match this.
    recommended_books.reverse()

    # The final output is a list containing the input book title
    # and a list of the 5 recommendations.
    return [book, recommended_books]


In [12]:
# --- Testing the Function ---
# Use the below code to test your function. The test logic is based on the problem requirements.
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! �🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! �🎉🎉🎉🎉
